# This notebook aims to develop a neural-network based classifier for the coffee bean dataset
## The technology of choice is pytorch

In [197]:
import torch
import pandas as pd
import os
from torchvision.io import read_image
from torch.utils.data import Dataset

bean_annotations = pd.read_csv("data/beans.csv")
bean_annotations.head()
DEFECT_CLASSES = dict(
    [
        (defect, index)
        for (index, defect) in enumerate(pd.unique(bean_annotations["defect_class"]))
    ]
)

In [198]:
from torchvision.transforms import v2

transforms = {
    "train": v2.Compose(
        [
            v2.Resize(size=(400, 400)),
            v2.RandomHorizontalFlip(),
            v2.RandomRotation(
                degrees=(20, 340), fill=(255, 255, 255)
            ),  # Augment the data with random rotations, setting the background to white
            v2.ToDtype(torch.float32, scale=True),
            v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    ),
    "test": v2.Compose(
        [
            v2.Resize(size=(400, 400)),
            v2.ToDtype(torch.float32, scale=True),
            v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    ),
}

In [199]:
class RoastDefectsDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.bean_annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.bean_annotations)

    def __getitem__(self, item):
        if torch.is_tensor(item):
            item = item.toList()

        img_name = self.bean_annotations.iloc[item, 0]
        img_dir = "-".join(img_name.split("-")[0:4])
        img_path = os.path.join(self.root_dir, img_dir, img_name)
        image = read_image(img_path)

        if self.transform:
            image = self.transform(image)

        annotations = DEFECT_CLASSES[
            self.bean_annotations.iloc[item, 1:]["defect_class"]
        ]
        return image, annotations

In [200]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(bean_annotations, train_size=0.9)

In [201]:
train.describe()

,img_name,origin_country,variety,processing_method,defect_class
count,2507,2507,2507,2507,2507
unique,2507,7,9,6,7
top,ecuador-typica-anaerobicNat-frag-1-6.png,ethiopia,caturra,washed,normal
freq,1,890,807,1532,1199


In [202]:
test.describe()

,img_name,origin_country,variety,processing_method,defect_class
count,279,279,279,279,279
unique,279,7,9,6,6
top,brazil-catuai-nat-quaker-2-10.png,ethiopia,caturra,washed,quaker
freq,1,87,90,158,118


In [203]:
from torch.utils.data import DataLoader, SubsetRandomSampler

data_train = RoastDefectsDataset(
    csv_file="data/beans.csv", root_dir="data/processed", transform=transforms["train"]
)

data_test = RoastDefectsDataset(
    csv_file="data/beans.csv", root_dir="data/processed", transform=transforms["test"]
)

train_sampler = SubsetRandomSampler(list(train.index))
test_sampler = SubsetRandomSampler(list(test.index))


train_loader = DataLoader(data_train, sampler=train_sampler, batch_size=64)
test_loader = DataLoader(data_test, sampler=test_sampler, batch_size=64)

dataloaders = {"test": test_loader, "train": train_loader}
dataset_sizes = {"test": len(test), "train": len(train)}

In [204]:
device = "mps"  # Train on GPU

In [205]:
from tempfile import TemporaryDirectory


def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, "best_model_params.pt")

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f"Epoch {epoch}/{num_epochs - 1}")
            print("-" * 10)

            # Each epoch has a training and validation phase
            for phase in ["train", "test"]:
                if phase == "train":
                    model.train()  # Set model to training mode
                else:
                    model.eval()  # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == "train"):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == "train":
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == "train":
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects / dataset_sizes[phase]
                print(f"Correct guesses in phase {phase}: {running_corrects}")
                print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

                # deep copy the model
                if phase == "val" and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        print(f"Best val Acc: {best_acc:4f}")

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model

In [206]:
from torchvision import models
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

model_ft = models.resnet18(weights="IMAGENET1K_V1")
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(DEFECT_CLASSES))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [207]:
model_ft = train_model(
    model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25
)

Epoch 0/24
----------
Correct guesses in phase train: 1476
train Loss: 1.1908 Acc: 0.5888
Correct guesses in phase test: 179
test Loss: 1.0577 Acc: 0.6416

Epoch 1/24
----------
Correct guesses in phase train: 1897
train Loss: 0.7586 Acc: 0.7567
Correct guesses in phase test: 198
test Loss: 0.8726 Acc: 0.7097

Epoch 2/24
----------
Correct guesses in phase train: 1979
train Loss: 0.6444 Acc: 0.7894
Correct guesses in phase test: 206
test Loss: 0.7373 Acc: 0.7384

Epoch 3/24
----------
Correct guesses in phase train: 2008
train Loss: 0.5765 Acc: 0.8010
Correct guesses in phase test: 217
test Loss: 0.6185 Acc: 0.7778

Epoch 4/24
----------
Correct guesses in phase train: 2058
train Loss: 0.5295 Acc: 0.8209
Correct guesses in phase test: 221
test Loss: 0.5825 Acc: 0.7921

Epoch 5/24
----------
Correct guesses in phase train: 2085
train Loss: 0.4844 Acc: 0.8317
Correct guesses in phase test: 226
test Loss: 0.5889 Acc: 0.8100

Epoch 6/24
----------
Correct guesses in phase train: 2122
train